# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [99]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034L,)
test label shape: (677L,)
dev label shape: (676L,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
/n


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [101]:
def P1(num_examples):
### STUDENT START ###
    for i in range(0,num_examples):
        print "example No: " + str(i+1)
        #show the label for the message
        print "label: " + str(train_labels[i])
        #show the category
        print "category: " + str(newsgroups_train.target_names[train_labels[i]])
        #show the text for the message
        print "text: " + str(train_data[i])+ "\n"
        print "--------------------------------------"

### STUDENT END ###
P1(5)

example No: 1
label: 1
category: comp.graphics
text: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

--------------------------------------
example No: 2
label: 3
category: talk.religion.misc
text: 

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.




(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [179]:
def P2():
### STUDENT START ###
    ########################################################
    #Part A
    #singleGram
    vectorizer = CountVectorizer()
    #bin words, 
    X = vectorizer.fit_transform(train_data)
    #Convert to matrix
    #X.toarray()
    print "Question 1a"
    print "The size of the vocabulary (single grams) is:  " + str(X.shape[1])
    
    print "The average number of non-zero features per example is: ",
    #calc missing values over total number of examples
    print X.nnz/float(X.shape[0]) #shows 196,700 words
    
    print "The fraction of the entries in the matrix that are non-zero is:",
    #Calc Sparsity 
    print str((X.nnz)/float(X.shape[0]*X.shape[1]))+ "\n"
    
    ########################################################
    #Part B
    print "Question 1b" 
    feature_names =vectorizer.get_feature_names()
    print "The 0th feature is: " + str((feature_names[0]))
    print "The last feature is: " + str(feature_names[-1]) +"\n"
    
    ########################################################
    #Part C
    print "Question 1c" 
    #Create new vocab list
    vocab = ["atheism", "graphics", "space", "religion"]
    #binarize it
    vocab_array = vectorizer.fit_transform(vocab) #.toarray()
    #apply bins to training dataset
    vocab_2_data = vectorizer.transform(train_data) #.toarray()
    #calc blank rate and overall shape
    print "The average number of non-zero features per example (with new 4 word vocab) is: ",
    print  str((vocab_2_data.nnz/float(vocab_2_data.shape[0]))) + "\n"
        
    ########################################################
    #Part D
    print "Question 1d" 
    #for words, Bi and TriGrams
    bigram_vectorizer = CountVectorizer(analyzer='char',ngram_range=(2, 3),token_pattern=r'\b\w+\b', min_df=1)

    #bin words and convert to matrix
    X_2 = bigram_vectorizer.fit_transform(train_data)#.toarray()
    X_2
    print "Bigrams and trigrams result in a vocabulary of size: " + str(X_2.shape[1]) + "\n"
    
    ########################################################
    #Part E
    print "Question 1e" 
    #for words, Bi and TriGrams
    bigram_vectorizer = CountVectorizer(min_df=10)

    #bin words and convert to matrix
    X_2 = bigram_vectorizer.fit_transform(train_data)#.toarray()
    X_2
    print "Words with at least 10 instances result in a vocabulary of size: " + str(X_2.shape[1]) + "\n"
    
    ########################################################
    #Part F
    print "Question 1f" 
    X = vectorizer.fit_transform(dev_data)
    dev_feature_names =vectorizer.get_feature_names()
    missing = 0
    for i in range(0,len(dev_feature_names)):
        if dev_feature_names[i] not in (feature_names):
            missing += 1
    print "The fraction of the words in the dev data  missing from the vocabulary is:  " + str(float(missing)/X.shape[1])
    
    ### STUDENT END ###
P2()

Question 1a
The size of the vocabulary (single grams) is:  26879
The average number of non-zero features per example is:  96.7059980334
The fraction of the entries in the matrix that are non-zero is: 0.00359782722696

Question 1b
The 0th feature is: 00
The last feature is: zyxel

Question 1c
The average number of non-zero features per example (with new 4 word vocab) is:  0.268436578171

Question 1d
Bigrams and trigrams result in a vocabulary of size: 35478

Question 1e
Words with at least 10 instances result in a vocabulary of size: 3064

Question 1f
4027
The fraction of the words in the dev data  missing from the vocabulary is:  0.247876400345


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [293]:
#def P3():
### STUDENT START ###
##################################################
#create n grams
#default CountVectorizer options 
#singleGram
vectorizer = CountVectorizer()
train_vec = vectorizer.fit_transform(train_data).toarray()
#train_vec.shape
#train_array = train_vec.toarray()
#print train_array [1:20,]
test_vec = vectorizer.transform(test_data)
#print test_vec.shape

##################################################
#Build KNN model
#report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier;
#find the optimal value for k. 
knn_model = KNeighborsClassifier()
n_neighbors = {'n_neighbors': [1, 3, 5, 7, 9, 15]} 
#sklearn library required a more explicit measure (ie f1 alone implies binary results and this is a multi class problem)
grid = GridSearchCV(estimator=knn_model, param_grid=n_neighbors, scoring = "f1_weighted")
grid.fit(train_vec, train_labels)

# summarize the results of the grid search
print "KNN best f1 score on Training Data: " + str(grid.best_score_)
print "KNN best nearest neighbor count: " + str(grid.best_estimator_.n_neighbors) + "\n"

##################################################
#Build MNB model
#Multinomial Naive Bayes model and find the optimal value for alpha. 
#gnb_model = BernoulliNB(binarize=0.5) #binarizer function is cutting off here!!
alphas = {'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
grid = GridSearchCV(estimator=clf_model, param_grid=alphas, scoring = "f1_weighted")
grid.fit(train_vec, train_labels)
#print(grid)
# summarize the results of the grid search
print "Multinomial NB Model best F1: " + str((grid.best_score_))
print "Multinomial NB Model best Alpha: " +str (grid.best_estimator_.alpha) + "\n"


clf_pred = clf_model.fit(train_vec, train_labels, alpha = grid.best_estimator_.alpha).predict(test_vec)
wrong_prediction = (clf_pred != test_labels)
print "Multinomial NB Model - Score on Test Pred: ",
print clf_model.metrics.f1_score


##################################################
#Build LR model
#fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization.

### STUDENT END ###
#P3()

KNN best f1 score on Training Data: 0.417778915182
KNN best nearest neighbor count: 15
KNN scores [0.41522664441317314, 0.39178746531123915, 0.41211272744086563, 0.40561404093953313, 0.41688792208238062, 0.41777891518161475]
k values [1, 3, 5, 7, 9]
Multinomial NB Model best F1: 0.828718198342


TypeError: cannot concatenate 'str' and 'float' objects

In [262]:
#gnb_model = BernoulliNB(binarize=0.5) #binarizer function is cutting off here!!
alphas = {'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
grid = GridSearchCV(estimator=clf_model, param_grid=alphas, scoring = "f1_weighted")
grid.fit(train_vec, train_labels)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

scores = [x[1] for x in grid.grid_scores_]
print 'scores',
print scores
scores_rounded = np.round(scores, 3)
print scores_rounded

print "\n"
df = []
df.append(scores_rounded)
df.append([0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0])
print np.array(df)

table = zip (np.round(scores, 3),[0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0] )
print table
print np.array(table)
#print grid.metrics.f1_score

GridSearchCV(cv=None, error_score='raise',
       estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='f1_weighted', verbose=0)
0.828718198342
0.01
scores [0.82551835418860098, 0.82618774680942975, 0.82871819834225602, 0.82295772554388091, 0.80961823895059748, 0.79662741908186041, 0.77195772065375345, 0.65530898520303482]
[ 0.826  0.826  0.829  0.823  0.81   0.797  0.772  0.655]


[array([ 0.826,  0.826,  0.829,  0.823,  0.81 ,  0.797,  0.772,  0.655])
 [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]]
[(0.82599999999999996, 0.0), (0.82599999999999996, 0.0001), (0.82899999999999996, 0.001), (0.82299999999999995, 0.01), (0.81000000000000005, 0.1), (0.79700000000000004, 0.5), (0.77200000000000002, 1.0), (0.65500000000000003, 2.0)]
[[  8.26000000e-01   0.000

In [291]:
#for i in scores:
scores_rounded = np.round_(scores, 3) 
print scores_rounded

print type(scores_rounded[2])
table = zip (scores_rounded,[0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0] )
print table
print "{0:.1f}%".format(np.array(table))

[ 0.826  0.826  0.829  0.823  0.81   0.797  0.772  0.655]
<type 'numpy.float64'>
[(0.82599999999999996, 0.0), (0.82599999999999996, 0.0001), (0.82899999999999996, 0.001), (0.82299999999999995, 0.01), (0.81000000000000005, 0.1), (0.79700000000000004, 0.5), (0.77200000000000002, 1.0), (0.65500000000000003, 2.0)]


ValueError: Unknown format code 'f' for object of type 'str'

In [297]:
#LR Model
regularization_c = {'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 0.9, 0.99,1,5,10,100]}
LR_model = LogisticRegression(penalty="l2")
grid = GridSearchCV(estimator=LR_model, param_grid=regularization_c, scoring = "f1_weighted")
grid.fit(train_vec, train_labels)
print(grid)
# summarize the results of the grid search
print "Logistic Regression best f1 score for training data: " + str(grid.best_score_)
print "Logistic Regression best regularization parameter: "+ str(grid.best_estimator_.C)


scores = [x[1] for x in grid.grid_scores_]
print 'scores',
print scores

print 'regularization_c',
print [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]

weights = [y[1] for y in grid.coef_]
print 'weights',
print weights

print 'regularization_c',
print [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]

#Logistic regression estimates a weight vector for each class, which you can access with the coef_ attribute. 
#Output the sum of the squared weight values for each class for each setting of the C parameter. 
#Briefly explain the relationship between the sum and the value of C.

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 0.9, 0.99, 1, 5, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='f1_weighted', verbose=0)
Logistic Regression best f1 score for training data: 0.767393454395
Logistic Regression best regularization parameter: 0.5
scores [0.49025589669234637, 0.65291334302019521, 0.72680331957672106, 0.76115762832299683, 0.76739345439454798, 0.76093867068647525, 0.75942242331090248, 0.75942242331090248, 0.75646671400119525, 0.7501871359976815, 0.74757550907094417]
regularization_c [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]


AttributeError: 'GridSearchCV' object has no attribute 'coef_'

In [307]:
def P3(regularization_c, accuracies):

### STUDENT START ###
    #iterate over different regularization weights
    for c_size in (regularization_c):
        #builds model
        LR_model = LogisticRegression(penalty="l2", C = c_size)
        LR_model.fit(train_vec, train_labels)
        
        #makes predictions on the dev dataset with the model we just built
        test_predicted_labels = LR_model.predict(test_vec)

        #accuracy Calcs
        wrong_prediction = (test_predicted_labels != test_labels)
        print "Training Sample Size:" + str(c_size), 
        accuracies.append(LR_model.coef_)
        #time_list.append(run_time)

### STUDENT END ###
regularization_c =  [0.0001, 0.001, 0.01, 0.1, 0.5, 0.9, 0.99,1,5,10,100]
accuracies = []
P3(regularization_c, accuracies)
print accuracies

Training Sample Size:0.0001 Training Sample Size:0.001 Training Sample Size:0.01 Training Sample Size:0.1 Training Sample Size:0.5 Training Sample Size:0.9 Training Sample Size:0.99 Training Sample Size:1 Training Sample Size:5 Training Sample Size:10 Training Sample Size:100 [array([[ -1.52533940e-03,  -1.18724621e-03,  -7.41337960e-05, ...,
         -4.49042447e-05,  -8.98084894e-05,  -3.63943428e-05],
       [  5.36222087e-04,  -1.84070945e-03,  -9.29400417e-05, ...,
         -4.10522482e-05,  -8.21044964e-05,   7.18034641e-05],
       [  6.53037643e-04,   3.06177763e-03,   9.30135863e-05, ...,
          4.93317908e-05,   9.86635817e-05,  -3.20281368e-05],
       [ -1.55153047e-03,  -2.11939984e-03,  -8.16528016e-05, ...,
         -3.74927861e-05,  -7.49855723e-05,  -3.47161226e-05]]), array([[ -7.37338972e-03,  -7.55560608e-05,  -3.35096792e-04, ...,
         -2.22302635e-04,  -4.44605270e-04,  -1.71376253e-04],
       [  3.51190392e-03,  -9.33029030e-03,  -9.17335124e-04, ...,
   

ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [6]:
#def P4():
### STUDENT START ###


### STUDENT END ###
#P4()

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [7]:
def empty_preprocessor(s):
    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

    
### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [8]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###

    

    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [11]:
#def P7():
    ### STUDENT START ###



    ### STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance